Article for inspiration: https://www.snowflake.com/blog/container-services-llama2-snowpark-ml/

# 0.0 Pre-Reqs

In [2]:
!pip uninstall snowflake-ml-python --y

Found existing installation: snowflake-ml-python 1.0.12
Uninstalling snowflake-ml-python-1.0.12:
  Successfully uninstalled snowflake-ml-python-1.0.12


In [3]:
!pip install snowflake-ml-python

  Obtaining dependency information for snowflake-ml-python from https://files.pythonhosted.org/packages/f4/20/aa43e61bf601a06b9b24b501ba9c6ac0bd9ebbc6d3679c6c597583518e25/snowflake_ml_python-1.0.12-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.4 MB/s eta 0:00:00a 0:00:01


In [7]:
from snowflake.snowpark.session import Session
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms

import os
import json
import sys
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [11]:
from snowflake.ml.model.models import llm

In [19]:
help(llm.LLM)

Help on class LLM in module snowflake.ml.model.models.llm:

class LLM(builtins.object)
 |  LLM(model_id_or_path: str, *, options: Optional[snowflake.ml.model.models.llm.LLMOptions] = None) -> None
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model_id_or_path: str, *, options: Optional[snowflake.ml.model.models.llm.LLMOptions] = None) -> None
 |      Args:
 |          model_id_or_path: model_id or local dir to PEFT lora weights.
 |          options: Options for LLM. Defaults to be None.
 |      
 |      Raises:
 |          ValueError: When unsupported.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  Mode = <enum 'Mode'>
 |      An

In [5]:
connection_parameters = json.load(open('/Users/skhara/Documents/Code/creds_spcs.json'))
session = Session.builder.configs(connection_parameters).create()

#### Compute Pool
A compute pool is a collection of virtual machines or nodes which can have GPUs.
This process takes time.

#### Learn More:
https://medium.com/snowflake/snowpark-container-services-a-tech-primer-99ff2ca8e741#:~:text=my_image%3Alatest-,Compute%20Pools,-A%20service%20in

In [6]:
session.sql('''CREATE COMPUTE POOL SKHARA_COMPUTE_GPU3
MIN_NODES = 1
MAX_NODES = 1
INSTANCE_FAMILY = "GPU_3"
''').collect()

SnowparkSQLException: (1304): 01b05f26-0001-e429-001c-dd8702443856: 002002 (42710): SQL compilation error:
Object 'SKHARA_COMPUTE_GPU3' already exists.

In [ ]:
session.sql(''

# 1.0 LLAMA Model Setup

## 1.1 Load LLAMA Model

In [6]:
HF_AUTH_TOKEN = "" #Your token from Hugging Face

In [7]:
# from transformers import pipeline
from snowflake.ml.model.models import huggingface_pipeline

llama_model = huggingface_pipeline.HuggingFacePipelineModel(task="text-generation",
                                                            model="meta-llama/Llama-2-7b-chat-hf",
                                                            token=HF_AUTH_TOKEN,
                                                            return_full_text=False,
                                                            max_new_tokens=100)

In [8]:
sys.getsizeof(llama_model)

48

## 1.2 Register the model

In [9]:
registry_name = 'SKHARA' #Replace this with the name of a database that you have access to
schema_name = 'BUILD_REGISTRY'

model_registry.create_model_registry(session= session,
                                     database_name= registry_name,
                                     schema_name= schema_name)

registry = model_registry.ModelRegistry(session= session,
                                        database_name= registry_name,
                                        schema_name= schema_name)

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


In [21]:
MODEL_NAME = "LLAMA2_MODEL_7b_CHAT"
MODEL_VERSION = "12"

llama_model_ref= registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

llama_model_ref

DataError: (0000) Model LLAMA2_MODEL_7b_CHAT/12 already exists. Unable to log the model.

## 1.3 Deploy Model

Pre-req: create a GPU compute pool

In [19]:
# Check if compute pool is ready. It may take some time for the compute resource to be ready.
session.sql('''Show compute pools like 'SKHARA_%';''').collect()

[Row(name='SKHARA_COMPUTE_GPU3', state='ACTIVE', min_nodes=1, max_nodes=1, instance_family='GPU_3', num_services=1, num_jobs=0, auto_suspend_secs=3600, auto_resume='true', active_nodes=1, idle_nodes=0, created_on=datetime.datetime(2023, 10, 31, 13, 28, 56, 790000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), resumed_on=datetime.datetime(2023, 11, 13, 8, 52, 44, 225000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), updated_on=datetime.datetime(2023, 11, 13, 9, 47, 41, 209000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), owner='SPC_USER_ROLE', comment=None),
 Row(name='SKHARA_COMPUTE_GPU5', state='SUSPENDED', min_nodes=1, max_nodes=1, instance_family='GPU_5', num_services=1, num_jobs=0, auto_suspend_secs=3600, auto_resume='true', active_nodes=0, idle_nodes=0, created_on=datetime.datetime(2023, 11, 1, 15, 20, 23, 967000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), resumed_on=datetime.datetime(2023, 11,

In [20]:
%%time
llama_model_ref.deploy(
    deployment_name="llama_predict",
    platform= deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    options={"compute_pool": "SKHARA_COMPUTE_GPU3",
             "num_gpus": 1
            },
    permanent = True,
)

TypeError: (0000) deploy() got an unexpected keyword argument 'force_image_build'

In [23]:
session.sql('show services').collect()

[]

# 2.0 Data Processing

We will load a JSON file to a Snowflake Table. For prediction purposes, we have two options - use Snowpark DataFrame or use Local Pandas DataFrame.

Snowpark dataframes allow us to work at scale and enable us to keep the data on server side without ever bringing data locally.

## 2.1 Load Data

In this lab, you do not have the required data in Snowflake so we will load a local .csv file to Snowflake using snowflake-snowpark library.

In [ ]:
json_dataset = pd.read_json("data/frosty_transcripts_all.jsonl", lines=True).convert_dtypes()
json_dataset.head()

In [ ]:
dict(json_dataset.iloc[0])

In [ ]:
TABLE_NAME = "BUILD_HOL_DATA"
session.write_pandas(json_dataset, table_name=TABLE_NAME, auto_create_table=True, overwrite=True)

## 2.2 Input: Prompt Engineering

In [ ]:
# I am going to bring 2 rows from the data to my local machine to create the prompt examples.
sdf_input = session.table('BUILD_HOL_DATA')
df_local = sdf_input.limit(2).to_pandas()
df_local.head()

In [ ]:
prompt_prefix = f'''[INST] <<SYS>>
Your output will be parsed by a computer program as a JSON object. Please respond ONLY with valid json that conforms to this JSON schema:
{{
  "name": {{
    "type": "string",
    "description": "The name of the person calling"
  }},
  "location": {{
    "type": "string",
    "description": "The name of the location where the person is calling from."
  }},
  "toy_list": {{
    "type": "array",
    "description": "The list of toys requested by the person calling."
  }},
  "required": ["name", "location", "toy_list"]
}}

Example 1:
Input: "{df_local['transcript'].iloc[0]}"
Output: {{"name": {df_local['name'].iloc[0]}, "location": {df_local['location'].iloc[0]}, "toy_list": {df_local['toy_list'].iloc[0]}}}

Example 2:
Input: "{df_local['transcript'].iloc[1]}"
Output: {{"name": {df_local['name'].iloc[1]}, "location": {df_local['location'].iloc[1]}, "toy_list": {df_local['toy_list'].iloc[1]}}}
<</SYS>>

Input:

'''

prompt_suffix = " [/INST]"

In [ ]:
import snowflake.snowpark.functions as F

input_df = sdf_input.with_column(
    '"inputs"',
    F.concat_ws(
        F.lit(" "), F.lit(prompt_prefix), F.col('"transcript"'), F.lit(prompt_suffix)
    ),
)

In [ ]:
input_df.write.mode("overwrite").save_as_table("DATA_WITH_PROMPT")

In [ ]:
# df_local['inputs'] = df_local['transcript'].apply(add_prompt)
# print(df_local['inputs'].iloc[3])

# 3.0 LLM Inference

## 3.1 Get Deployed Model

In [ ]:
REGISTRY_NAME = 'SKHARA'
SCHEMA_NAME = 'BUILD_REGISTRY'
MODEL_NAME = 'LLAMA2_MODEL_7b_CHAT'
MODEL_VERSION = '7'
DEPLOYMENT_NAME = 'llama_predict'

In [ ]:
registry = model_registry.ModelRegistry(session= session,
                                        database_name= REGISTRY_NAME,
                                        schema_name= SCHEMA_NAME)

In [ ]:
model_list = registry.list_models()
model_list.to_pandas()

In [ ]:
model_list = registry.list_deployments(model_name = MODEL_NAME, model_version = MODEL_VERSION)
model_list.to_pandas()

In [ ]:
model = model_registry.ModelReference(registry=registry, model_name=MODEL_NAME, model_version=MODEL_VERSION)

## 3.2 Inference using LLMs

In [ ]:
sdf_data_prompt = session.table('DATA_WITH_PROMPT')
sdf_data_prompt.limit(2).to_pandas()

In [ ]:
session.sql('SHOW COMPUTE POOLS').collect()

In [ ]:
res = model.predict(
    deployment_name= DEPLOYMENT_NAME,
    data= sdf_data_prompt
)

In [ ]:
df_local = res.limit(5).to_pandas() #bring 5 rows locally

## 3.3 Output Processing
Ensure that processing code conforms to the JSON Structure provided during Prompt Engineering.

In [ ]:
import json

def format_output(output_string):
    try:
        # Step 1: Parse the outer string to convert it to a list of dictionaries
        outer_list = json.loads(output_string)
        
        # Step 2: Extract the 'generated_text' value from the first dictionary in the list
        generated_text_str = outer_list[0]['generated_text']
        
        # Step 3: Locate the JSON object within the 'generated_text' value
        start_pos = generated_text_str.find('{')
        end_pos = generated_text_str.rfind('}')
        if start_pos == -1 or end_pos == -1:
            raise ValueError("No JSON object found in generated_text")
        json_str = generated_text_str[start_pos:end_pos + 1]
        
        # Step 4: Parse the JSON object to convert it to a dictionary
        json_obj = json.loads(json_str)
        
        return json_obj
    except:
        return 'Could not parse output'

In [ ]:
for i in range(len(df_local)):
    print(f'\n\n **** Transcript # {i} ****')
    print(df_local['transcript'].iloc[i])
    print('\n')
    print(format_output(df_local['outputs'].iloc[i]))

# 4.0 Clean Up

In [ ]:
model.delete_deployment(deployment_name=DEPLOYMENT_NAME)

In [ ]:
session.sql("ALTER COMPUTE POOL IF EXISTS SKHARA_COMPUTE_GPU3 STOP ALL").collect()